In [2]:
import pysam as ps
import numpy as np
import collections
import matplotlib.pyplot as plt
from numpy.core.defchararray import array, center
import pysam as ps
import numpy as np
from collections import defaultdict
from scipy.ndimage.measurements import label, standard_deviation
from scipy.stats.mstats_basic import kstest, normaltest
from sklearn.cluster import KMeans
import sys
from scipy.signal import savgol_filter
import math
from subprocess import call
import os.path
from utils import Gene, TSS, Point
from scipy import stats
from sklearn import svm
import sympy
import math
from math import e
import random
from keras import Model
from keras.models import load_model
import numpy as np
import os
import pandas as pd

In [3]:
dic = {
    'gene':'./data/gene/GRCh37.gene.bed',
    'non_gene':'./data/gene/non_gene.bed',
    'non_gene_4':'./data/gene/non_gene_4.bed',
    'non_gene_2':'./data/gene/non_gene_2.bed',
    'fasta':'/home/jiay/Desktop/hg19/hg19.fa',
    'bam1':'./data/051.bam',
    'bam2':'./data/051_2.bam',
    'bam3':'./data/051_3.bam',
    'bam123':'./data/051_123.bam',
    'bam4':'./data/051_4.bam',
    'bam5':'./data/051_5.bam',
    'TSS_low':'./data/gene/low_expressed.bed',
    'TSS_HK':'./data/gene/HK.bed',
    'TSS_silent':'./data/gene/silent_gene_TSS.bed',
    'ATAC_hema_1':'./data/gene/ATAC_hema_1.bed',
    'ATAC_hema_2':'./data/gene/ATAC_hema_2.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed',
    'ATAC_Bcell':'./data/gene/ATAC_Bcell.bed',
    'ATAC_Brain':'./data/gene/ATAC_Brain.bed'
    }
TSS_HK = []
with open(dic['TSS_HK'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        if ll[0] in ['1','2','3']:
            TSS_HK.append(TSS(ll[0], int(int(ll[1])+1000)))
# with open(dic['ATAC_hema_1'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         TSS_HK.append(TSS(ll[0], int((int(ll[1])+int(ll[2]))/2)))
        


TSS_NonGene = []
with open(dic['non_gene'],'r') as f:
    for line in f:
        ll = line.strip().split('\t')
        TSS_NonGene.append(TSS(ll[0], int(ll[1])+int(1000)))
# with open(dic['ATAC_Bcell'],'r') as f:
#     for line in f:
#         ll = line.strip().split('\t')
#         if ll[0] in ['1','2','3'] and ll[2] == '0':
#             TSS_NonGene.append(TSS(ll[0],int(ll[1])))

bamfile = ps.AlignmentFile(dic['bam123'],'rb')

TSS_NonGene = TSS_NonGene[:len(TSS_HK)]

TSSes_x = TSS_HK + TSS_NonGene
TSSes_y = [1]*len(TSS_HK) + [0]*len(TSS_NonGene)
perm = random.sample(range(len(TSSes_x)),len(TSSes_x))
TSSes_x = np.array(TSSes_x)
TSSes_y = np.array(TSSes_y)
TSSes_x = TSSes_x[perm[:len(perm)]]
TSSes_y = TSSes_y[perm[:len(perm)]]
labels = TSSes_y
up = 1000
down = 1000

In [15]:
# #lstm
# feature_matrix = []
# win = 120
# for j, tss in enumerate(TSSes_x):
#     chrom = tss.chrom
#     start = tss.pos - up
#     end = tss.pos + down
#     up_end = np.zeros(up+down, dtype= int)
#     down_end = np.zeros(up+down, dtype= int)
#     long = np.zeros(up+down, dtype= int)
#     short = np.zeros(up+down, dtype= int)
#     cov = np.zeros(up+down, dtype= int)
#     wps = np.zeros(up+down, dtype=float)
#     for r in bamfile.fetch(chrom[-1], start-500, end + 500):
#         if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
#             if r.reference_start + abs(r.isize) < start:
#                 continue
#             if r.reference_start >= end:
#                 continue
#             ss = r.reference_start - start
#             ee = r.reference_start - start + abs(r.isize)
#             if ss >= 0:
#                 up_end[ss] += 1
#             else:
#                 ss = 0
#             if ee < end - start:
#                 down_end[ee] += 1
#             else:
#                 ee = end - start
#             for i in range(ss, ee):
#                 cov[i] += 1
#             if 200 >= abs(r.isize) > 130:
#                 for i in range(ss, ee):
#                     long[i] += 1
#             if abs(r.isize) <= 130:
#                 for i in range(ss, ee):
#                     short[i] += 1
#             # wps_total
#             region1 = int(max(0, ss + win/2))
#             region2 = int(min(ee - win/2, end-start))
#             i = region1
#             while i < region2:
#                 wps[i] += 1
#                 i = i+1
#             # wps_part
#             region1 = int(max(0, ss - win/2))
#             region2 = int(min(end-start, ss + win/2))
#             i = region1
#             while i < region2:
#                 wps[i] -= 1
#                 i = i + 1
#             # wps_part
#             region1 = int(max(ee - win/2, 0))
#             region2 = int(min(ee + win/2, end-start))
#             i = region1
#             while i < region2:
#                 wps[i] -= 1
#                 i = i+1
#     # k = 0
#     # win = 40
#     # feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
#     # while k < (up+down)/win:
#     #     ss = k * win
#     #     ee = k * win + win
#     #     ff = []
#     #     ff.append(int(round(np.mean(cov[ss:ee]))))
#     #     ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
#     #     ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
#     #     ff.append(int(round(np.mean(wps[ss:ee]))))
#     #     feature_win[k] = np.array(ff)
#     # feature_matrix.append(feature_win)
#     ff = [cov,long-short,abs(up_end-down_end),wps]
#     feature_matrix.append([[row[i]for row in ff] for i in range(len(ff[0]))])
# feature_matrix = np.array(feature_matrix)
# lstm_x = feature_matrix
#lstm
win = 40
feature_matrix = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
    up_end = np.zeros(up+down, dtype= int)
    down_end = np.zeros(up+down, dtype= int)
    long = np.zeros(up+down, dtype= int)
    short = np.zeros(up+down, dtype= int)
    cov = np.zeros(up+down, dtype= int)
    wps = np.zeros(up+down, dtype=float)
    win = 120
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            ss = r.reference_start - start
            ee = r.reference_start - start + abs(r.isize)
            if ss >= 0:
                up_end[ss] += 1
            else:
                ss = 0
            if ee < end - start:
                down_end[ee] += 1
            else:
                ee = end - start
            for i in range(ss, ee):
                cov[i] += 1
            if 200 >= abs(r.isize) > 130:
                for i in range(ss, ee):
                    long[i] += 1
            if abs(r.isize) <= 130:
                for i in range(ss, ee):
                    short[i] += 1
            # wps_total
            region1 = int(max(0, ss + win/2))
            region2 = int(min(ee - win/2, end-start))
            i = region1
            while i < region2:
                wps[i] += 1
                i = i+1
            # wps_part
            region1 = int(max(0, ss - win/2))
            region2 = int(min(end-start, ss + win/2))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i + 1
            # wps_part
            region1 = int(max(ee - win/2, 0))
            region2 = int(min(ee + win/2, end-start))
            i = region1
            while i < region2:
                wps[i] -= 1
                i = i+1
    k = 0
    win = 40
    feature_win = np.zeros((int((up+down)/win), 4), dtype= int)
    while k < (up+down)/win:
        ss = k * win
        ee = k * win + win
        ff = []
        ff.append(int(round(np.mean(cov[ss:ee]))))
        ff.append(int(round(np.mean(long[ss:ee]-short[ss:ee]))))
        ff.append(int(round(np.sum(abs(up_end[ss:ee]-down_end[ss:ee])))))
        ff.append(int(round(np.mean(wps[ss:ee]))))
        feature_win[k] = np.array(ff)
        k = k + 1
    feature_matrix.append(feature_win)
feature_matrix = np.array(feature_matrix)

lstm_x = []
for mat in feature_matrix:
    lstm_x.append(mat)
lstm_x = np.array(lstm_x)



In [4]:
'''cnn_input'''
raw_data = []
for j, tss in enumerate(TSSes_x):
    chrom = tss.chrom
    start = tss.pos - up
    end = tss.pos + down
#    distribution_matrix = np.zeros((int(up+down), 200), dtype=int)
    distribution_matrix = np.zeros((200,200),dtype=int)
    for r in bamfile.fetch(chrom[-1], start-500, end + 500):
        if (not r.is_reverse) and (not r.is_unmapped) and (not r.mate_is_unmapped) and r.mate_is_reverse and 50 < abs(r.isize) < 250:
            if r.reference_start + abs(r.isize) < start:
                continue
            if r.reference_start >= end:
                continue
            if r.reference_start < start:
                continue
            if r.reference_start + abs(r.isize) > end:
                continue
            ss = max(0, r.reference_start - start)
            relative_isize = abs(r.isize)-50
            distribution_matrix[ss//10,relative_isize] += 1
    raw_data.append(distribution_matrix)
raw_data = np.array(raw_data)

cnn_x = []
for mat in raw_data:
    cnn_x.append(mat)
cnn_x = np.array(cnn_x)

In [25]:
print(lstm_x.shape,cnn_x.shape)

(1638, 50, 4) (1638, 200, 200)


In [ ]:
from keras import layers
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

cnn_x = cnn_x.reshape(cnn_x.shape[0], cnn_x.shape[1], cnn_x.shape[2], 1)

'''lstm'''
lstm_input = layers.Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
x = layers.Convolution1D(filters=64, kernel_size=2, strides=2, activation='relu', padding='same')(lstm_input)
x = layers.AveragePooling1D(pool_size=2, strides=2)(x)
x = layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(x)
x = layers.AveragePooling1D(pool_size=2, strides=2)(x)
x = layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(x)
x = layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True))(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu', kernel_regularizer=None, bias_regularizer=None)(x)

'''cnn'''
cnn_input = layers.Input(shape=(cnn_x.shape[1], cnn_x.shape[2],1), name='cnn_input')
y = layers.Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(cnn_input)
y = layers.Activation('relu')(y)
y = layers.MaxPool2D(strides=(2,2),pool_size=(2,2))(y)
y = layers.Conv2D(filters=32,kernel_size=3,strides=(1,1),padding='same')(y)
y = layers.Activation('relu')(y)
y = layers.MaxPool2D(strides=(2,2),pool_size=(2,2))(y)
y = layers.Flatten()(y)
y = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)

z = layers.concatenate([x,y])

z = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(z)
z = layers.Dense(50,activation='relu',kernel_regularizer=None, bias_regularizer=None)(z)

output = layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(z)

model = Model(inputs=[lstm_input,cnn_input], outputs=output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit([lstm_x,cnn_x], labels, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./conv_lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

In [7]:
from keras import layers
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

'''cnn_1D'''
cnn_input = layers.Input(shape=(cnn_x.shape[1], cnn_x.shape[2]), name='cnn_input')
y = layers.Convolution1D(filters=64, kernel_size=2, strides=2, activation='relu', padding='same')(cnn_input)
y = layers.MaxPooling1D(pool_size=2, strides=2)(y)
y = layers.Dropout(0.2)(y)
y = layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(y)
y = layers.MaxPooling1D(pool_size=2, strides=2)(y)
y = layers.Flatten()(y)
y = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(50,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)
output = layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(y)

model = Model(inputs=cnn_input, outputs=output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(cnn_x, labels, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./cnn_1D.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/300
12/12 - 1s - loss: 0.6883 - accuracy: 0.5122 - val_loss: 0.6903 - val_accuracy: 0.4634
Epoch 2/300
12/12 - 0s - loss: 0.6877 - accuracy: 0.5115 - val_loss: 0.6900 - val_accuracy: 0.4634
Epoch 3/300
12/12 - 0s - loss: 0.6847 - accuracy: 0.5115 - val_loss: 0.6899 - val_accuracy: 0.4634
Epoch 4/300
12/12 - 0s - loss: 0.6814 - accuracy: 0.5156 - val_loss: 0.6888 - val_accuracy: 0.4634
Epoch 5/300
12/12 - 1s - loss: 0.6795 - accuracy: 0.5115 - val_loss: 0.6876 - val_accuracy: 0.4634
Epoch 6/300
12/12 - 0s - loss: 0.6751 - accuracy: 0.5190 - val_loss: 0.6842 - val_accuracy: 0.4634
Epoch 7/300
12/12 - 0s - loss: 0.6718 - accuracy: 0.5319 - val_loss: 0.6811 - val_accuracy: 0.4634
Epoch 8/300
12/12 - 0s - loss: 0.6666 - accuracy: 0.5407 - val_loss: 0.6764 - val_accuracy: 0.4634
Epoch 9/300
12/12 - 0s - loss: 0.6599 - accuracy: 0.5739 - val_loss: 0.6709 - val_accuracy: 0.5000
Epoch 10/300
12/12 - 0s - loss: 0.6518 - accuracy: 0.5984 - val_loss: 0.6646 - val_accuracy: 0.5122
Epoch 11/

In [17]:
print(cnn_x.shape)

(1638, 200, 200, 1)


In [12]:
from keras import layers
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint
layers.av
cnn_x = cnn_x.reshape(cnn_x.shape[0], cnn_x.shape[1], cnn_x.shape[2], 1)
'''cnn'''
cnn_input = layers.Input(shape=(cnn_x.shape[1], cnn_x.shape[2],1), name='cnn_input')
y = layers.Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(cnn_input)
y = layers.Activation('relu')(y)
y = layers.MaxPool2D(strides=(2,2),pool_size=(2,2))(y)
y = layers.Dropout(0.2)(y)
y = layers.Conv2D(filters=64,kernel_size=3,strides=(1,1),padding='same')(y)
y = layers.Activation('relu')(y)
y = layers.MaxPool2D(strides=(2,2),pool_size=(2,2))(y)
y = layers.Dropout(0.2)(y)
# y = layers.Conv2D(filters=32,kernel_size=3,strides=(1,1),padding='same')(y)
# y = layers.Activation('relu')(y)
# y = layers.MaxPool2D(strides=(2,2),pool_size=(2,2))(y)
y = layers.Flatten()(y)
y = layers.Dense(200,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)
y = layers.Dropout(0.2)(y)
y = layers.Dense(50,activation='relu',kernel_regularizer=None, bias_regularizer=None)(y)

output = layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(y)

model = Model(inputs=cnn_input, outputs=output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(cnn_x, labels, epochs=10, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./cnn.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

Epoch 1/10
12/12 - 84s - loss: 0.6809 - accuracy: 0.5102 - val_loss: 0.6668 - val_accuracy: 0.4756
Epoch 2/10
12/12 - 79s - loss: 0.6121 - accuracy: 0.6872 - val_loss: 0.5536 - val_accuracy: 0.8537
Epoch 3/10
12/12 - 79s - loss: 0.4463 - accuracy: 0.8677 - val_loss: 0.3606 - val_accuracy: 0.8841
Epoch 4/10
12/12 - 81s - loss: 0.3158 - accuracy: 0.8847 - val_loss: 0.2836 - val_accuracy: 0.9146
Epoch 5/10
12/12 - 80s - loss: 0.2370 - accuracy: 0.9166 - val_loss: 0.3284 - val_accuracy: 0.9024
Epoch 6/10
12/12 - 81s - loss: 0.2173 - accuracy: 0.9199 - val_loss: 0.2699 - val_accuracy: 0.9207
Epoch 7/10
12/12 - 82s - loss: 0.1960 - accuracy: 0.9362 - val_loss: 0.2490 - val_accuracy: 0.9207
Epoch 8/10
12/12 - 82s - loss: 0.1788 - accuracy: 0.9410 - val_loss: 0.2501 - val_accuracy: 0.9268
Epoch 9/10
12/12 - 78s - loss: 0.1658 - accuracy: 0.9512 - val_loss: 0.2442 - val_accuracy: 0.9268
Epoch 10/10
12/12 - 84s - loss: 0.1549 - accuracy: 0.9505 - val_loss: 0.2606 - val_accuracy: 0.9329


In [16]:
from keras import layers
from tensorflow import keras
from keras.callbacks import EarlyStopping, ModelCheckpoint

'''lstm'''
lstm_input = layers.Input(shape=(lstm_x.shape[1], lstm_x.shape[2]), name='lstm_input')
# x = layers.Convolution1D(filters=64, kernel_size=2, strides=2, activation='relu', padding='same')(lstm_input)
# x = layers.AveragePooling1D(pool_size=2, strides=2)(x)
# x = layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(x)
# x = layers.AveragePooling1D(pool_size=2, strides=2)(x)
# x = layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same')(x)
x = layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True))(lstm_input)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
x = layers.Dense(100, activation='relu', kernel_regularizer=None, bias_regularizer=None)(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100,activation='relu',kernel_regularizer=None, bias_regularizer=None)(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(50,activation='relu',kernel_regularizer=None, bias_regularizer=None)(x)

output = layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None,name='6')(x)

model = Model(inputs=lstm_input, outputs=output)

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
history = model.fit(lstm_x, labels, epochs=300, batch_size=128, validation_split=0.1,callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./lstm.h5', save_best_only=True)],
                        shuffle=True, verbose=2)

/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/300
12/12 - 44s - loss: 0.6355 - accuracy: 0.6255 - val_loss: 0.5209 - val_accuracy: 0.7805
Epoch 2/300
12/12 - 1s - loss: 0.4984 - accuracy: 0.7890 - val_loss: 0.3628 - val_accuracy: 0.9329
Epoch 3/300
12/12 - 0s - loss: 0.3603 - accuracy: 0.8758 - val_loss: 0.2793 - val_accuracy: 0.9329
Epoch 4/300
12/12 - 0s - loss: 0.2855 - accuracy: 0.9104 - val_loss: 0.2536 - val_accuracy: 0.9329
Epoch 5/300
12/12 - 0s - loss: 0.2559 - accuracy: 0.9104 - val_loss: 0.2546 - val_accuracy: 0.9268
Epoch 6/300
12/12 - 0s - loss: 0.2237 - accuracy: 0.9193 - val_loss: 0.2497 - val_accuracy: 0.9268
Epoch 7/300
12/12 - 0s - loss: 0.2155 - accuracy: 0.9281 - val_loss: 0.2466 - val_accuracy: 0.9390
Epoch 8/300
12/12 - 0s - loss: 0.2059 - accuracy: 0.9308 - val_loss: 0.2529 - val_accuracy: 0.9390
Epoch 9/300
12/12 - 0s - loss: 0.2144 - accuracy: 0.9288 - val_loss: 0.2397 - val_accuracy: 0.9390
Epoch 10/300
12/12 - 0s - loss: 0.1969 - accuracy: 0.9342 - val_loss: 0.2327 - val_accuracy: 0.9329
Epoch 11

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = keras.Sequential()
model.add(keras.layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same',input_shape=(lstm_x.shape[1], lstm_x.shape[2])))
model.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
model.add(keras.layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same'))
model.add(keras.layers.AveragePooling1D(pool_size=2, strides=2))
model.add(keras.layers.Convolution1D(filters=32, kernel_size=2, strides=2, activation='relu', padding='same'))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(300, activation='relu', kernel_regularizer=None, bias_regularizer=None))
model.add(keras.layers.Dropout(0.2))
# model.add(keras.layers.Dense(16, activation='relu', kernel_regularizer=None, bias_regularizer=None))
# model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=None, bias_regularizer=None))

# print(model.summary())

adam = keras.optimizers.Adam(lr=1e-4)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history = model.fit(lstm_x, labels, epochs=300, batch_size=128, validation_split=0.1,
                        callbacks=[EarlyStopping(patience=10), ModelCheckpoint(filepath='./' + 'lstm_4_feature.h5', save_best_only=True)],
                        shuffle=True, verbose=2)


/home/jiay/.local/lib/python3.8/site-packages/keras/optimizer_v2/optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
2021-11-24 16:26:46.719403: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/300
12/12 - 4s - loss: 0.6182 - accuracy: 0.6621 - val_loss: 0.4494 - val_accuracy: 0.7500
Epoch 2/300
12/12 - 1s - loss: 0.3820 - accuracy: 0.8691 - val_loss: 0.2869 - val_accuracy: 0.9024
Epoch 3/300
12/12 - 1s - loss: 0.2896 - accuracy: 0.9050 - val_loss: 0.2477 - val_accuracy: 0.9146
Epoch 4/300
12/12 - 1s - loss: 0.2619 - accuracy: 0.9111 - val_loss: 0.2333 - val_accuracy: 0.9085
Epoch 5/300
12/12 - 1s - loss: 0.2428 - accuracy: 0.9172 - val_loss: 0.2100 - val_accuracy: 0.9207
Epoch 6/300
12/12 - 1s - loss: 0.2361 - accuracy: 0.9186 - val_loss: 0.2090 - val_accuracy: 0.9268
Epoch 7/300
12/12 - 1s - loss: 0.2160 - accuracy: 0.9308 - val_loss: 0.2183 - val_accuracy: 0.8963
Epoch 8/300
12/12 - 1s - loss: 0.2154 - accuracy: 0.9349 - val_loss: 0.2172 - val_accuracy: 0.9024
Epoch 9/300
12/12 - 1s - loss: 0.2213 - accuracy: 0.9288 - val_loss: 0.1942 - val_accuracy: 0.9207
Epoch 10/300
12/12 - 1s - loss: 0.2014 - accuracy: 0.9376 - val_loss: 0.1992 - val_accuracy: 0.9268
Epoch 11/